In [ ]:
# pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 7.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm

from collections import Counter

In [ ]:
def delete_punctuation_marks_tag(word):
    word = word.replace(",","")
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace("!","")
    word = word.replace(":","")
    word = word.replace("?","")
    word = word.replace("~","")
    word = word.replace("\\","")
    word = word.replace("\"","")
    word = word.replace(";","")
    return word.lower()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
eat_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/translated_eat_info.csv')
eat_review = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/translated_eat_review.csv')


In [ ]:
eat_info

,name,address,category,price,rvw_cnt,rating,p_id,reviews
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,5000,72,4.5,553,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...
1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,5000,94,4.2,383,서울특별시 광진구 구의2동에 위치한 서북면옥에서 물냉면(9천원)과 접시만두 6조각(...
2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,5000,40,4.0,78,기본에 충실해서 더 맛있었던 핫도그였습니다. 들어가서 음식이 나오기까지는 꽤 시간이...
3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,5000,13,4.0,715,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다..1위가...
4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,5000,9,4.0,45,혼밥하기 좋은곳.김밥도 괜찮고 잔치국수도 깔끔하니 괜찮다.. 잔치국수와 건대김밥 주...
...,...,...,...,...,...,...,...,...
841,마우로아,서울특별시 광진구 능동로 320 1F,칵테일 / 와인,35000,10,4.2,225,성수 유명한 맛집에서 이제는 군자 맛집으로너무 가보고싶었던 곳이라 엄마랑 동생이랑 ...
842,글래시,서울특별시 광진구 능동로 23-14 1F,칵테일 / 와인,35000,19,3.5,81,뚝섬역에서 가까운 글래시 와인바에요 ! 와인바 찾던 중 우연히 발견했는데 가격도 맛...
843,리바,서울특별시 광진구 워커힐로 177 비스타 워커힐 1F,칵테일 / 와인,50000,4,NaN,215,제주도에서 하는 빛의 벙커를 서울에서도 한다고 하여워커힐 방문!빛의 시어터 다 보고...
844,몽,서울특별시 광진구 아차산로33길 25,칵테일 / 와인,50000,1,NaN,270,분위기 어둑지고 아늑한 곳. 인도풍? 칵테일바 인데 분수도 있고 신발벗고 편히 앉아...


In [ ]:
eat_review

,res_name,user_name,review,rating,p_id,u_id
0,원조할아버지손두부,달려요달려,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,5,553,715
1,원조할아버지손두부,ultraus,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,5,553,453
2,원조할아버지손두부,세상의 모든 음식을 정복하겠어,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,5,553,1080
3,원조할아버지손두부,조선의장희빈,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,5,553,1479
4,원조할아버지손두부,미콩,드디어 먹어본 아차산 순두부!! 같이 가 준 친구들이 이미 전에도 맛본 경험이 있어...,5,553,954
...,...,...,...,...,...,...
5137,리바,나뮤,소리 너무너무너무 시끄러워요. 클럽마냥 소리질러야 대화가 됩니다~ 야경은 예뻐요,0,215,655
5138,리바,Consumericus,비비드아워로 방문.종류는 아주 다양하다거나 신라 정도의 퀄리티는 아니지만 뷰가 예쁘...,3,215,58
5139,리바,썸머쏭쏭,"• 딱! 괜찮은 정도에요• 물론, 뷰+분위기는 굉장히 힙하나,• 팥빙수는 정말.. ...",3,215,1158
5140,몽,Tiffajy,분위기 어둑지고 아늑한 곳. 인도풍? 칵테일바 인데 분수도 있고 신발벗고 편히 앉아...,5,270,256


In [ ]:
eat_review_by_user = pd.DataFrame(index = eat_review['u_id'].unique(), columns=['reviews'])

for id in tqdm(eat_review['u_id'].unique()):
    review_all = ''
    for review in eat_review.loc[eat_review['u_id']==id,'review']:
        if type(review) != float:
            review_all += review+'. '
    eat_review_by_user.loc[id,'reviews'] = review_all

  0%|          | 0/1730 [00:00<?, ?it/s]

In [ ]:
eat_info['tags']=''
eat_review_by_user['tags']=''

In [ ]:
eat_info.head()

,name,address,category,price,rvw_cnt,rating,p_id,reviews,tags
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,5000,72,4.5,553,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,
1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,5000,94,4.2,383,서울특별시 광진구 구의2동에 위치한 서북면옥에서 물냉면(9천원)과 접시만두 6조각(...,
2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,5000,40,4.0,78,기본에 충실해서 더 맛있었던 핫도그였습니다. 들어가서 음식이 나오기까지는 꽤 시간이...,
3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,5000,13,4.0,715,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다..1위가...,
4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,5000,9,4.0,45,혼밥하기 좋은곳.김밥도 괜찮고 잔치국수도 깔끔하니 괜찮다.. 잔치국수와 건대김밥 주...,


In [ ]:
eat_review_by_user.head()

,reviews,tags
715,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,
453,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,
1080,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,
1479,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,
954,드디어 먹어본 아차산 순두부!! 같이 가 준 친구들이 이미 전에도 맛본 경험이 있어...,


In [ ]:
# with open('../data/stop_words.txt','r',-1,'utf-8') as f:
#     stop_words = set(f.read().split('/'))

In [ ]:

stopwords = ['맛','더','것','곳','가격','때','밥','진짜','정도','건대','집','여기','음식','좀','처음','수',
             '생각','사람','방문','주문','정말','친구','바로','다른','가게','바','이','조금','또',
             '내','로우','거','하나','메뉴','요리','그','점','가지','별로','식당','한번','추천','삼선','보고','그냥','꽤','보통','나','입','제','직원',
             '다시','말','왜','그릇','무게','꼭','다음','개','느낌','아주','자리','거리','때문','리뷰',
             '중','도','웨이','팅','일단','사진','저','계속','가장','제일','걸','듯','안','건',
             '명','간이','이유','역시','편이','추가','편','재','먹기','거의','시간','기도','테이블','분','마',
             '담','날','근처','손','땐','못','완전','애정','항상','첫','제대로','입맛','이번','먹음',
             '궁합','최고','원래','가면','예전','뭔가','평점','만','한국','다만','기대','배','막','오위','시작',
             '종류','후','냄새','중간','해먹','번','뒤','배달',
             '그램','점점','갑자기','감','살짝','물','콤','색',
             '간','사실','강추','최애','자체','이전','카','골목','음식점','상태','금방','위생','용서','조리','질','요','그때',
             '널','리지','약간','앞','시로','마루','무난','기본','취향','깨','양','줄','수도','개인','매장','식사','두',
             '생','산다','사을','자극','다소','둘','전체','끼','서비스','이이','맛집','아부','얼','비주','오픈','저녁','으;',
             '마끼','대기','운','재료','퀄리티','점심','소스','포장','내부','할머니','토마','식감','동네','일','혼자','요즘','어마','가득','해',
             '저희','볼','인기','뭐','말씀','평일','세','도착','사이즈','일반','밖','간장','후회','인분','손님','달','자주','함',
             '굳이','모든','매우','거기','인생','학교','크기','분위기','절대','주변','무조건','인정','푹','식','옆','주차','위치','존맛','스타일',
             '이모','무엇','밤','새벽','길','기준','알','자꾸','전형','무척','차','베이스','불어','임','위','음','이제','얘기','이해','비','기분',
             '지금','덜','답','북적','신장','개업','끝','방송','위해','법','특유','모습','비교','부분','웨이팅','옥','은','동이','팀','베','부',
             '하의','의사','케이','쿠라','인테리어','마당','대비','인','워낙','쿠니','반','데','마음','개조','미리','단점','하이','','','','',
             '양념','가성','구의역','술집','양도','세트','직접','시징','가족','체인점','안나',###
             '이집','저집','적','시','물이','모두','갓','글','수가','집의','합리','부집','준','해도','갈이','','','','','','',
             '','','','','','','','','','','','','','','','','','','','','','','','','','',
             '','','','','','','','','','','','','','','','','','','','','','','','','',
             '','','','','','','','','','','','','','','','','','','','','','','','','','','','','',
             ]

In [ ]:
# pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 54.9 MB/s 


In [ ]:
import konlpy
Okt = konlpy.tag.Okt()

def get_tags_to_txt(text):
    Okt_morphs = Okt.pos(str(text)) 
    

    Noun_words = []
    for word, pos in Okt_morphs:
        if pos == 'Noun':
            Noun_words.append(word)
    
    unique_Noun_words = set(Noun_words)
    for word in unique_Noun_words:
        if word in stopwords:
            while word in Noun_words: Noun_words.remove(word)  # 최종결과 : Noun_word
    
    if type(text) != str:
        return ''
    tags = Noun_words
    
    
    tags = ','.join(tags)
    
    return tags

In [ ]:
for i in tqdm(eat_info.index):
    eat_info.loc[i,'tags'] = get_tags_to_txt(eat_info.loc[i,'reviews'])
    print(get_tags_to_txt(eat_info.loc[i,'reviews']))
    
for i in tqdm(eat_review_by_user.index):
    eat_review_by_user.loc[i,'tags'] = get_tags_to_txt(eat_review_by_user.loc[i,'reviews'])
    print(get_tags_to_txt(eat_review_by_user.loc[i,'reviews']))

  0%|          | 0/846 [00:00<?, ?it/s]

i번째 초당,순두부,순두부,향,이집,전혀,아차산,등산,필수,코스,부집,늘,회전,십분,이상,적,시,두부,물이,두부,순두부,콩국수,콩국수,콩국,물이,순두부,두부,모두,갓,글,질감,젓갈,별미,막걸리,한잔,수가,두부,집의,매력,포인트,막걸리,막걸리,반찬,두부,합리,등산,기,유기농,두부,순환,아차산,등산,서울,시내,풍경,눈,하산,할아버지,부집,두부,모,새우,젓갈,고추,두부,젓갈,향,고추,가루,호불호,사이드,순두부,두부,두부,순두부,순두부,양으로,요거트,타,순두부,두유,술,안주,지평막걸리,요구르트,처럼,톡,부랑,조화,콩국수,기회,아차산,순두부,준,전,경험,등산,해도,두부,부가,일인,순두부,막걸리,두부,두부,백설기,두부,등산,퍼센트,두부,수록,순두부,주시,멸치젓,갈이,달달,두부,할아버지,두부,순두부,공기밥,막걸리,새우젓,아차산,겨울,콩국수,이서,두부,순두부,공기,부랑,순두부,저번,공기밥,히트,천,원,주제,무채,순두부,쯤,무채,막걸리,알밤,막걸리,아차산,막걸리,아차산,막걸리,아차산,아차산,막걸리,강,콩,비지,공짜,아차산역,할아버지,두부,아차산,등산,시원,막걸리,잔,전,택시,타고,등산,여부,순두부,두부,두부,와이프,순두부,동의,순두부,공기밥,스윽스윽,젓갈,두부,파시,두부,삭,김치,막걸리,잔,캬,뜻,막걸리,구비,아침,아침,마침,아차산,막걸리,한잔,두부,두부,아침,두부,기계,소리,윙윙,낭만,아차산,등산,막걸리,잔,립스틱,자국,그대로,헹구,고춧가루,잔,두부,비위,상해,지역,외,정보,아침,일찍,순두부,허옇,두부,냄비,떡,당황,했닼,맘,등산,막걸리,그땐,전이,도토리묵,의아,이상,주관,슴,순두부,무생채,액젓,과,명일,두부,순두부,공기밥,요식업,지배,단어,조합,아시,원조,클리셰,할아버지,출동,아차산,아차,두부,할아버지,두부,소개,두부,관심,갱생,조,증말,기회,그동안,아차산,등산,객,등산,막걸리,한잔,한잔,순두부,모양새,뿌리,젓갈,흐트러트리,유아,퇴행,조,콩,달달,순두부,교도소,도입,갱생,경구,투입,다가,두부,교도소,벽,다가,가끔,갱생,마려우,탈옥,숟가락,퍼먹다,죄,범죄,율,대한민국,전혀,

  0%|          | 0/1730 [00:00<?, ?it/s]

i번째 초당,순두부,순두부,향,이집,전혀,동치미,향,맛뒤,육수,혼밥,김밥,잔치국수,쫄깃함신맛,익,동치미,역대,최악,돈코츠,라멘,실패,어디,서도,보지,스프,카레,어찌,고민,서서,속도
i번째 아차산,등산,필수,코스,부집,늘,회전,십분,이상,적,시,두부,물이,두부,순두부,콩국수,콩국수,콩국,물이,순두부,두부,모두,갓,글,질감,젓갈,별미,막걸리,한잔,수가,육수,고명,커버,오천원,아래쪽,핫도그,깜짝,스테이크,감자,튀김,회,냉면,매콤,달콤,맵,촐,갈이,냉면집,면,평냉,뚝뚝,이로,회심,냉면,함흥,냉면,갈매기살,갈매기살,여름,휴가,여름,전화,가세,소바,우동,겨울,시즌,우동,자루,소바,면,튀김,류,선,듬,튀김,해결,면발,쫄깃,밀가루,남,소바,우동,겨울,시즌,우동,자루,소바,면,튀김,류,선,듬,튀김,해결,면발,쫄깃,밀가루,남,학생,초밥,가기,호야,탕수육,짬뽕,마파두부,등,중식,술,한잔,메인,맥주,등,볶음,인상,짬뽕,국물,뒤끝,꽉,농후,국물,녹두,당면,쫄깃함,마리,새우,신선,글함,비도,젠,평균,이상,접근성,꼬치,염려,꼬치,실력,프랑스,전반,정성,디저트,류,꿀맛,사이,고민,찍기,퀄,소시지,선택,단백질,야채,동시,충족,점수,초딩,데리,가기,낮,토요일,영업,영양성분,헌터,조건,소시지,제외,샐러드,통실,햄,소시지,샐러드,소시,와플,토핑,서북,냉면,갖,좌석,수가,지하,공부,거나,화장실,관리,사용,커피,크레이프,케이크,쌈,마이,플레이,장소,가끔,인원,근방,가기,그니,쉬,라떼,우유,여러가지,블렌딩해,우유,단맛,강조,시럽,단맛,존재,시럽,띠용,곳곳,유머,감각,재미,인상,정성,잔,커피,사운드,음악,잠시,정신,적당,아파트,촌,광장,면,개방,인스타,휴무,확인,토요일,휴무,움,과자류,기회
i번째 두부,집의,매력,포인트,막걸리,막걸리,반찬,두부,합리,갑,스시,마카,만칠천원,입성,마카,진행,스시,마카,꼬,다리,삼이일,땡,외치,입문,무랴,입성,중국,화가,중국집,랄,허겁지겁,중국,장사,평가,그릭,요거트,무리,그릭,요거트,제철,과일,토핑,캐모마일,시럽,브런치,도전,똠양꿍,똠양꿍,팟타이,새우,튀김,오징어,순위,

In [ ]:
eat_info

,name,address,category,price,rvw_cnt,rating,p_id,reviews,tags
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,5000,72,4.5,553,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,"초당,순두부,순두부,향,이집,전혀,아차산,등산,필수,코스,부집,늘,회전,십분,이상,..."
1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,5000,94,4.2,383,서울특별시 광진구 구의2동에 위치한 서북면옥에서 물냉면(9천원)과 접시만두 6조각(...,"서울특별시,광진구,구의,동,서북,면옥,물냉면,과,접시,만두,물냉면,육수,슴슴,소고기..."
2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,5000,40,4.0,78,기본에 충실해서 더 맛있었던 핫도그였습니다. 들어가서 음식이 나오기까지는 꽤 시간이...,"핫도그,문,준비,고려,핫도그,특성,아쉬움,소세지,굽기,적,당선,양은,간식,광진구,투..."
3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,5000,13,4.0,715,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다..1위가...,"온,첫날,짜장면,망플,장안,분식집,사장,대한민국,요식업,친절,장안,떡볶이,튀김,오뎅..."
4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,5000,9,4.0,45,혼밥하기 좋은곳.김밥도 괜찮고 잔치국수도 깔끔하니 괜찮다.. 잔치국수와 건대김밥 주...,"혼밥,김밥,잔치국수,잔치국수,김밥,잔치국수,다대기,움,김밥,계란,당근,단무지,위주,..."
...,...,...,...,...,...,...,...,...,...
841,마우로아,서울특별시 광진구 능동로 320 1F,칵테일 / 와인,35000,10,4.2,225,성수 유명한 맛집에서 이제는 군자 맛집으로너무 가보고싶었던 곳이라 엄마랑 동생이랑 ...,"성수,군자,엄마,동생,와인,가야,양파,스프,꼬기,더욱,군자역,거주,인구,군자,집들이..."
842,글래시,서울특별시 광진구 능동로 23-14 1F,칵테일 / 와인,35000,19,3.5,81,뚝섬역에서 가까운 글래시 와인바에요 ! 와인바 찾던 중 우연히 발견했는데 가격도 맛...,"뚝섬역,글,래시,와인,와인,발견,세번,파스타,시중,사장,면,관건,파스타,와인,와인,..."
843,리바,서울특별시 광진구 워커힐로 177 비스타 워커힐 1F,칵테일 / 와인,50000,4,NaN,215,제주도에서 하는 빛의 벙커를 서울에서도 한다고 하여워커힐 방문!빛의 시어터 다 보고...,"제주도,빛,벙커,서울,워커힐,빛,시어터,워커힐,뷰,말모,층,쪽,보틀,일정,금액,이상..."
844,몽,서울특별시 광진구 아차산로33길 25,칵테일 / 와인,50000,1,NaN,270,분위기 어둑지고 아늑한 곳. 인도풍? 칵테일바 인데 분수도 있고 신발벗고 편히 앉아...,"어둑지,인도,칵테일,분수,신발,아지트,실제,불빛,촛불,칵테일,여자,격의,상콤,달달,..."


In [ ]:
eat_info.to_csv('/content/drive/MyDrive/Colab Notebooks/data/eat_info.csv')

In [ ]:
eat_review_by_user

,reviews,tags
715,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,"초당,순두부,순두부,향,이집,전혀,동치미,향,맛뒤,육수,혼밥,김밥,잔치국수,쫄깃함신..."
453,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,"아차산,등산,필수,코스,부집,늘,회전,십분,이상,적,시,두부,물이,두부,순두부,콩국..."
1080,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,"두부,집의,매력,포인트,막걸리,막걸리,반찬,두부,합리,갑,스시,마카,만칠천원,입성,..."
1479,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,"등산,기,유기농,두부,순환,아차산,등산,서울,시내,풍경,눈,하산,할아버지,부집,두부..."
954,드디어 먹어본 아차산 순두부!! 같이 가 준 친구들이 이미 전에도 맛본 경험이 있어...,"아차산,순두부,준,전,경험,등산,해도,두부,부가,일인,순두부,막걸리,두부,두부,백설..."
...,...,...
537,오픈매장인거 보고 왔다갔어요!개인적으로 쉬림프파스타 정말 맛있었어요!!또 들를래요 ...,"쉬,림프,파스타"
1597,오픈한지 별로 안된 가게라서 가게에 사람들이 많이 없더라구요...사실 들어가기 두려...,"한지,남자,둘이서,로마,휴일,흑백영화,빔,프로젝트,노래,술,취하,주시,레스토랑,겸,..."
603,자양동에 이런 분위기 와인펍이 들어올줄은 몰랐네요 엄청 고급집니다. 소개에보면 국내...,"자양동,와인펍,고급,소개,국내,유명,쉐프,개발,참여,와인,고급,레스토랑,비쥬"
655,소리 너무너무너무 시끄러워요. 클럽마냥 소리질러야 대화가 됩니다~ 야경은 예뻐요.,"소리,클럽,마냥,소리,대화,야경"


In [ ]:
eat_review_by_user.to_csv('/content/drive/MyDrive/Colab Notebooks/data/eat_review_by_user.csv')

In [ ]:
# pip install gensim

In [ ]:
# w2v = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
import gensim
w2v = gensim.models.Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/ko/ko.bin')

In [ ]:
All_tags = set()
for tags in eat_info['tags']:
    All_tags.update(set(tags.split(',')))

In [ ]:
new_df = pd.DataFrame()
for word in All_tags:
    try:
        f_vec = w2v.wv.get_vector(word)
        new_df[word] = f_vec
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """


In [ ]:
new_df

,쿡,도넛,소생,메이,건축,가재,참새,화이트,서울대,프로그램,...,알코올,야마,당근,스탈,거나,존경,가산,선입견,집착,장마
0,-0.321912,0.137143,0.722951,-0.294999,-0.839302,0.739670,0.723060,-1.231490,-0.161435,-1.513309,...,2.579754,-0.187728,0.969753,-0.517097,-0.227632,-1.999859,1.136156,-0.055855,0.327600,-0.531187
1,-0.576380,-0.063783,-0.128207,-2.982411,0.855411,-0.560193,-0.527000,-1.070189,0.401992,-0.458278,...,0.267901,-2.432376,-0.419091,-0.520162,2.257966,-0.534731,-0.148673,0.110761,-0.229832,-0.533982
2,0.610621,-0.114523,0.772327,1.632749,0.868524,0.145369,-0.096612,0.044919,1.455974,-1.112888,...,0.722778,0.526291,0.410699,0.220960,-1.857827,3.174925,-1.379611,0.101719,0.629122,0.612103
3,0.376098,0.229198,0.015730,2.667686,-0.958006,0.419144,0.463202,1.132294,0.261728,3.873265,...,1.268141,1.389950,0.307008,-0.200452,-0.360986,-2.277550,-0.142628,-0.118905,-0.839456,-0.324770
4,-0.369545,0.084575,-0.495065,-2.510468,-0.920211,-0.323437,-0.543576,-0.608402,-0.925093,1.853934,...,-0.226680,-0.475267,0.283989,-0.628805,-0.708224,0.288826,-0.560024,-0.124916,-1.204548,-0.027643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-0.292975,0.010550,0.241571,-1.269906,-0.262587,-0.206651,-0.289930,-0.143936,0.831147,0.098770,...,0.793858,-1.488747,-0.250451,-0.677384,1.515267,1.844635,0.915974,0.141869,-0.541788,0.335876
196,-0.413033,0.544986,0.106726,1.477107,3.011224,0.113148,0.385412,-0.221835,-0.729493,1.405153,...,0.342183,-0.580578,0.508922,-0.162143,0.168176,-0.437600,0.313467,0.208282,0.586995,0.266921
197,-0.038669,0.216006,-0.403715,1.193433,1.118008,-0.120339,-0.454347,-0.702938,-0.919159,2.265103,...,2.149317,-0.357886,0.431717,0.355998,-0.495177,0.539690,-0.042352,0.421940,1.106519,0.465500
198,-0.751390,0.117568,1.348373,-0.504781,1.989186,-0.221537,-0.110594,-0.887419,0.608835,0.592697,...,0.851668,-0.702976,0.496913,0.450891,0.211185,-1.239224,0.809776,-0.400455,-0.144603,-0.418246


In [ ]:
x = new_df.T
cluster_count=120
model = KMeans(n_clusters=cluster_count)
model.fit(x)
model.predict(x)
Y=x.copy()
Y['kmeans_id'] = model.predict(x)

In [ ]:
km = list(Y['kmeans_id'])
max_count = 0
max_count_id = ''
for i in range (0, cluster_count):
    if km.count(i) > max_count:
        max_count = km.count(i);
        max_count_id = i;
    print(i,': ', km.count(i))
max_count_id

0 :  50
1 :  30
2 :  53
3 :  50
4 :  37
5 :  75
6 :  1
7 :  15
8 :  16
9 :  53
10 :  71
11 :  13
12 :  37
13 :  41
14 :  31
15 :  64
16 :  45
17 :  53
18 :  31
19 :  23
20 :  24
21 :  82
22 :  290
23 :  7
24 :  7
25 :  37
26 :  24
27 :  34
28 :  57
29 :  842
30 :  55
31 :  19
32 :  23
33 :  3
34 :  1
35 :  27
36 :  55
37 :  1
38 :  15
39 :  27
40 :  59
41 :  6
42 :  19
43 :  1
44 :  72
45 :  12
46 :  2
47 :  104
48 :  32
49 :  2
50 :  54
51 :  43
52 :  46
53 :  14
54 :  16
55 :  20
56 :  37
57 :  11
58 :  31
59 :  48
60 :  27
61 :  226
62 :  1
63 :  35
64 :  319
65 :  62
66 :  4
67 :  2
68 :  44
69 :  81
70 :  2
71 :  20
72 :  12
73 :  49
74 :  194
75 :  50
76 :  1
77 :  25
78 :  88
79 :  6
80 :  17
81 :  76
82 :  125
83 :  12
84 :  18
85 :  2
86 :  31
87 :  3
88 :  67
89 :  1
90 :  2
91 :  1
92 :  42
93 :  36
94 :  68
95 :  3
96 :  30
97 :  6
98 :  3
99 :  4
100 :  22
101 :  74
102 :  68
103 :  2
104 :  7
105 :  3
106 :  1
107 :  228
108 :  6
109 :  1
110 :  239
111 :  54
112 :  53
11

29

In [ ]:
km_df = pd.DataFrame()
km_df['group_'+str(max_count_id)] = pd.Series(list(Y[Y['kmeans_id'] == max_count_id].index))
for i in range(0, cluster_count):
    col_name = 'group_'+str(i)
    words = list(Y[Y['kmeans_id'] == i].index)
    km_df[col_name] = pd.Series(words)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
km_df.fillna(0)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# remove groups we don't need
# rm_groups = [17, 26, 50, 63]
# for i in rm_groups:
#     group_name = 'group_'+str(i);
#     stop_words.update(set(km_df[group_name].dropna()))
# with open('../data/stop_words.txt','w',-1,'utf-8') as f:
#     f.write('/'.join(stop_words))

In [ ]:
eat_info.set_index(eat_info['p_id'],inplace=True,drop=True)
del eat_info['p_id']
eat_info.head()

,name,address,category,price,rvw_cnt,rating,reviews,tags
p_id,,,,,,,,
553,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,5000,72,4.5,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,"초당,순두부,순두부,향,이집,전혀,아차산,등산,필수,코스,부집,늘,회전,십분,이상,..."
383,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,5000,94,4.2,서울특별시 광진구 구의2동에 위치한 서북면옥에서 물냉면(9천원)과 접시만두 6조각(...,"서울특별시,광진구,구의,동,서북,면옥,물냉면,과,접시,만두,물냉면,육수,슴슴,소고기..."
78,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,5000,40,4.0,기본에 충실해서 더 맛있었던 핫도그였습니다. 들어가서 음식이 나오기까지는 꽤 시간이...,"핫도그,문,준비,고려,핫도그,특성,아쉬움,소세지,굽기,적,당선,양은,간식,광진구,투..."
715,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,5000,13,4.0,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다..1위가...,"온,첫날,짜장면,망플,장안,분식집,사장,대한민국,요식업,친절,장안,떡볶이,튀김,오뎅..."
45,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,5000,9,4.0,혼밥하기 좋은곳.김밥도 괜찮고 잔치국수도 깔끔하니 괜찮다.. 잔치국수와 건대김밥 주...,"혼밥,김밥,잔치국수,잔치국수,김밥,잔치국수,다대기,움,김밥,계란,당근,단무지,위주,..."


In [ ]:
eat_info.to_csv('./eat_info_tags.csv')

In [ ]:
res_matrix = pd.DataFrame(index=eat_info.index,columns=list(km_df.columns))
user_matrix = pd.DataFrame(index=eat_review_by_user.index,columns=list(km_df.columns))
res_matrix = res_matrix.fillna(0)
user_matrix = user_matrix.fillna(0)

In [ ]:
res_matrix.head()

,group_29,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
p_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
553,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
383,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
715,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
user_matrix.head()

,group_29,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
715,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
453,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1479,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
954,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
for id in tqdm(res_matrix.index):
    tags = Counter(eat_info.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                res_matrix.loc[id,'group_'+str(num)] += tags[tag]

  0%|          | 0/846 [00:00<?, ?it/s]

In [ ]:
res_matrix.head()

,group_29,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
p_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
553,261,12,8,2,18,6,17,0,6,2,4,8,0,10,28,19,22,21,11,11,8,5,10,50,1,1,2,3,11,4,19,7,3,3,0,3,66,1,9,7,9,0,1,0,37,0,1,21,15,2,12,8,8,21,9,2,5,6,7,9,6,36,2,9,135,13,0,1,11,36,0,0,1,13,361,6,0,14,6,3,1,47,33,27,1,2,6,3,27,0,0,0,19,3,6,0,6,2,0,0,7,46,15,0,1,0,3,38,0,0,27,13,23,0,0,5,8,0,2,34
383,100,13,12,2,8,4,6,0,1,1,5,9,2,10,42,10,10,22,6,8,5,3,21,22,5,0,0,0,14,14,16,9,0,1,0,1,95,0,1,4,1,0,2,0,23,1,1,7,18,0,11,4,95,1,5,2,4,2,9,7,3,34,0,10,52,31,0,0,15,35,0,0,0,9,346,8,1,2,7,1,6,99,35,7,0,0,8,0,13,0,0,2,3,8,18,0,9,2,0,0,5,31,8,2,0,1,2,35,0,0,14,6,24,0,0,4,7,0,1,26
78,25,6,1,7,1,1,2,0,0,0,4,2,0,1,8,2,8,4,3,3,2,1,9,6,1,0,0,1,5,5,4,0,0,0,0,0,55,0,0,0,2,0,0,0,2,1,0,12,1,0,2,0,0,0,0,0,0,0,3,2,0,14,0,3,10,6,1,0,2,5,0,0,0,2,40,3,0,2,10,0,1,12,6,1,0,0,0,0,11,0,0,0,0,1,2,0,1,0,0,0,1,5,5,0,0,0,0,5,1,0,7,3,7,0,0,2,0,0,0,2
715,30,7,2,1,15,2,1,0,0,3,2,7,4,2,6,2,9,4,3,2,3,0,13,11,1,2,0,3,4,4,2,1,0,0,0,0,27,3,3,0,0,0,2,0,10,0,0,3,11,1,17,1,1,5,2,2,0,0,1,1,1,7,1,0,14,5,0,0,5,11,0,0,0,5,106,7,1,0,3,0,4,9,12,2,1,1,0,0,3,0,0,0,8,1,3,0,4,0,0,0,3,3,5,0,0,1,0,5,0,0,5,2,7,0,0,12,4,0,1,10
45,5,1,0,0,0,0,0,0,0,0,0,1,0,0,2,0,3,3,0,1,0,0,1,2,0,0,0,0,1,0,0,1,0,0,0,0,4,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,3,0,0,0,2,2,0,0,0,0,43,0,0,0,2,0,1,7,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,5,0,0,0,0,0,0,0,0,5


In [ ]:
for id in tqdm(user_matrix.index):
    tags = Counter(eat_review_by_user.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                user_matrix.loc[id,'group_'+str(num)] += tags[tag]


In [ ]:
user_matrix.head()

,group_29,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
715,1,0,0,0,0,0,0,0,0,0,1,0,0,1,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
453,8,2,1,0,2,1,1,0,0,0,1,2,0,1,3,1,3,1,1,2,1,1,3,7,0,0,1,0,3,1,4,2,0,0,0,0,12,0,0,0,1,1,0,0,2,0,0,4,0,0,0,0,3,0,0,0,0,1,4,2,0,3,0,0,2,0,0,0,1,8,0,1,0,1,40,1,0,0,2,1,1,5,5,0,0,0,1,0,2,0,0,0,1,2,1,0,3,0,0,0,2,0,0,0,0,0,0,2,0,0,1,3,7,0,0,0,0,0,0,2
1080,6,0,0,2,5,1,0,0,0,0,0,0,3,0,3,3,1,0,2,0,0,0,0,1,0,0,0,0,2,0,0,1,0,0,0,0,6,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,6,0,0,1,0,0,0,2,0,0,0,0,0,13,0,0,0,0,0,1,1,5,0,1,0,0,0,1,0,0,0,0,0,1,0,2,0,0,0,0,2,2,0,0,0,0,2,0,0,3,0,0,0,0,0,0,0,0,0
1479,3,0,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,5,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,4,1,0,0,0,1,0,0,0,0,14,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,3,0,0,2,1,4,0,0,1,0,0,0,0
954,18,2,0,0,11,1,11,0,1,0,5,2,1,2,11,1,3,6,1,0,1,0,11,7,0,0,0,0,0,5,3,2,0,0,0,0,41,2,4,3,2,3,0,0,5,0,0,11,1,6,4,2,1,0,1,0,1,2,3,5,4,4,0,5,9,3,0,0,0,5,0,3,0,2,33,10,1,2,4,0,0,14,7,1,0,1,0,0,3,0,0,0,3,2,1,0,5,0,0,0,2,2,2,0,0,0,0,5,0,0,16,3,3,0,0,0,1,0,0,4


In [ ]:
km_df.count()

group_29     842
group_0       50
group_1       30
group_2       53
group_3       50
group_4       37
group_5       75
group_6        1
group_7       15
group_8       16
group_9       53
group_10      71
group_11      13
group_12      37
group_13      41
group_14      31
group_15      64
group_16      45
group_17      53
group_18      31
group_19      23
group_20      24
group_21      82
group_22     290
group_23       7
group_24       7
group_25      37
group_26      24
group_27      34
group_28      57
group_30      55
group_31      19
group_32      23
group_33       3
group_34       1
group_35      27
group_36      55
group_37       1
group_38      15
group_39      27
group_40      59
group_41       6
group_42      19
group_43       1
group_44      72
group_45      12
group_46       2
group_47     104
group_48      32
group_49       2
group_50      54
group_51      43
group_52      46
group_53      14
group_54      16
group_55      20
group_56      37
group_57      11
group_58      

In [ ]:
eat_review_score = pd.DataFrame(eat_review, columns=['u_id','p_id','rating'])
eat_review_score.sort_values(by=['u_id'], axis=0,inplace=True)
eat_review_score.reset_index(drop = True, inplace=True)
eat_review_score.head()

,u_id,p_id,rating
0,0,542,3
1,0,287,3
2,0,163,0
3,0,775,3
4,0,310,0


In [ ]:
res_matrix.to_csv('/content/drive/MyDrive/Colab Notebooks/data/res_matrix.csv')
user_matrix.to_csv('/content/drive/MyDrive/Colab Notebooks/data/user_matrix.csv')
eat_review_score.to_csv('/content/drive/MyDrive/Colab Notebooks/data/score_board.csv',index=False)
km_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/eat_cluster.csv',index=False)
km_df.count().to_csv('/content/drive/MyDrive/Colab Notebooks/data/group_count.csv')